In [1]:
#installing required packages

! pip install transformers

! pip install --upgrade transformers

! pip install datasets

! conda install pytorch torchvision torchaudio cpuonly -c pytorch3

You should consider upgrading via the '/Users/philippounds/opt/anaconda3/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/philippounds/opt/anaconda3/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/philippounds/opt/anaconda3/bin/python -m pip install --upgrade pip' command.

UnavailableInvalidChannel: The channel is not accessible or is invalid.
  channel name: pytorch3
  channel url: https://conda.anaconda.org/pytorch3
  error code: 404

You will need to adjust your conda configuration to proceed.
Use `conda config --show channels` to view your configuration's current state,
and use `conda config --show-sources` to view config file locations.




In [3]:
import glob

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

from datasets import load_dataset, load_metric, Dataset
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import torch


import random

In [46]:
fileList = []

for i in range(7):
    file = ""
    x = i+1
    file = open("data/text"+str(x)+".txt","r")
    file = file.read()
    fileList.append(file)
    

    

df = data = {'text':  [fileList[0], fileList[1],fileList[2], fileList[3], fileList[4], fileList[5], fileList[6]],
        'label': ['profile', 'nonprofile', 'nonprofile','nonprofile', 'nonprofile', 'nonprofile', 'profile'],
        'filename':  ['SILK PARACHUTE', 'Draft No. 4', 'Dah Dah Doo Dah Dah Dah Dah Dah Doo Dah La Ti Mi Fa La So Fa Mi', 'Phi Beta Football', 'Direct Eye Contact', 'The Orange Trapper', 'The Patch']
         }

full_df = pd.DataFrame (df, columns = ['text','label', 'filename'])


label_dict = {0:'profile', 1:'nonprofile'}

splits = [0]*6 + [1]*1
full_df['split'] = pd.Series(splits).sample(frac=1, random_state=2323).reset_index(drop=True)
display(full_df.head())

print(full_df.shape)

,text,label,filename,split
0,SHOUTS & MURMURS about the author's memories o...,profile,SILK PARACHUTE,0
1,Block. It puts some writers down for months. I...,nonprofile,Draft No. 4,0
2,Do I remember when I had my first drink? Absol...,nonprofile,Dah Dah Doo Dah Dah Dah Dah Dah Doo Dah La Ti ...,0
3,"At a reception in Castine, Maine, years ago, I...",nonprofile,Phi Beta Football,0
4,"Fishing from a canoe in the Delaware River, I ...",nonprofile,Direct Eye Contact,0


(7, 4)


In [47]:
# tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neo-2.7B")
tokenizer.name_or_path

'EleutherAI/gpt-neo-2.7B'

In [51]:
train_encodings = tokenizer(full_df.query('split==0')['text'].tolist(), truncation=True)
val_encodings = tokenizer(full_df.query('split==1')['text'].tolist(), truncation=True)

In [52]:
no_classes = len(full_df.query('split==0')['label'].unique())

print(no_classes)
train_classes = [label_dict[class_ind] for class_ind in range(no_classes)]

2


In [53]:
#Create custom Datasets Class
class ArticlesDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, label):
        self.encodings = encodings
        self.labels = label

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['label'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

#Create datasets from encodings
train_dataset = ArticlesDataset(train_encodings, full_df.query('split==0')['label'].tolist())
val_dataset = ArticlesDataset(val_encodings, full_df.query('split==1')['label'].tolist())

In [ ]:
#model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", id2label=label_dict, num_labels=2)
model = AutoModelForSequenceClassification.from_pretrained("EleutherAI/gpt-neo-2.7B", id2label=label_dict, num_labels=2)
model.name_or_path

In [ ]:
training_args = TrainingArguments("test_trainer",
                                 logging_strategy='epoch')
training_args

In [ ]:
metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
trainer.evaluate(train_dataset)

In [ ]:
trainer.evaluate(val_dataset)

In [ ]:
trainer.predict(train_dataset)

In [ ]:
trainer.evaluate(val_dataset)

In [ ]:
trainer.save_model('mcphee-writing-gpt2.7')

In [ ]:
from transformers import pipeline

mag_classifier = pipeline( task='text-generation', model='mcphee-writing-gpt2.7')

max_tok_len = 512 #mag_classifier.tokenizer.max_model_input_sizes[tokenizer.name_or_path]
#mag_classifier(full_df['text'].tolist())

res = mag_classifier([text[:max_tok_len] for text in full_df['text'].tolist()])